In [8]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [9]:
## Load dataset
data_path = "../gmm_dataset_c20k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

BATCH_SIZE = 20
NUM_EPOCHS = 1000
LEARNING_RATE = 5 * 1e-4
CUDA = torch.cuda.is_available()
PATH = 'baseline-mlp-2round-%dsamples' % (SAMPLE_SIZE)
DEVICE = torch.device('cuda:0')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D)

In [10]:
from local_enc import *
from global_oneshot import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_z.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_eta, enc_z)

In [12]:
(NUM_EPOCHS, NUM_DATASETS, S, B, CUDA, device, path) = Train_Params
SubTrain_Params = (device, S, B) + Model_Params

NUM_BATCHES = int((NUM_DATASETS / B))

indices = torch.randperm(NUM_DATASETS)
for step in range(1):
    optimizer.zero_grad()
    batch_indices = indices[step*B : (step+1)*B]
    obs = Data[batch_indices]
    obs = shuffler(obs).repeat(S, 1, 1, 1)
    if CUDA:
        obs =obs.cuda().to(device)

In [13]:
(device, sample_size, batch_size, N, K, D) = SubTrain_Params
obs_tau, obs_mu, state, log_w_f_z, q_eta, p_eta, q_z, p_z = Init_step_eta(models, obs, N, K, D, sample_size, batch_size)

In [17]:
q_z['zs'].log_prob.shape

torch.Size([10, 20, 60])

In [4]:
from os_ep import *
train_baseline(models, EP, optimizer, Data, Model_Params, Train_Params)

epoch: 0\1000 (7s),  eubo: -537.644,  elbo: -636.374,  ess: 1.103
epoch: 1\1000 (7s),  eubo: -348.188,  elbo: -364.218,  ess: 1.426
epoch: 2\1000 (7s),  eubo: -300.987,  elbo: -315.976,  ess: 1.494
epoch: 3\1000 (7s),  eubo: -281.840,  elbo: -294.993,  ess: 1.588
epoch: 4\1000 (7s),  eubo: -279.344,  elbo: -290.922,  ess: 1.643
epoch: 5\1000 (7s),  eubo: -278.679,  elbo: -289.474,  ess: 1.661
epoch: 6\1000 (7s),  eubo: -278.328,  elbo: -288.643,  ess: 1.686
epoch: 7\1000 (7s),  eubo: -278.147,  elbo: -288.131,  ess: 1.708
epoch: 8\1000 (7s),  eubo: -278.003,  elbo: -287.785,  ess: 1.719
epoch: 9\1000 (7s),  eubo: -277.870,  elbo: -287.524,  ess: 1.726
epoch: 10\1000 (7s),  eubo: -277.747,  elbo: -287.283,  ess: 1.733
epoch: 11\1000 (7s),  eubo: -277.663,  elbo: -287.101,  ess: 1.755
epoch: 12\1000 (7s),  eubo: -277.539,  elbo: -286.854,  ess: 1.753
epoch: 13\1000 (7s),  eubo: -277.531,  elbo: -286.750,  ess: 1.772
epoch: 14\1000 (7s),  eubo: -277.479,  elbo: -286.634,  ess: 1.769
epoch

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/hao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-8351e0914ddb>", line 2, in <module>
    train_baseline(models, EP, optimizer, Data, Model_Params, Train_Params)
  File "../training.py", line 161, in train_baseline
    loss.backward()
  File "/home/hao/anaconda3/lib/python3.7/site-packages/torch/tensor.py", line 107, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/home/hao/anaconda3/lib/python3.7/site-packages/torch/autograd/__init__.py", line 93, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._

KeyboardInterrupt: 

In [5]:
torch.save(enc_z.state_dict(), '../weights/enc-z-%s' % PATH)
torch.save(oneshot_eta.state_dict(), '../weights/oneshot-eta-%s' % PATH)

In [6]:
def test(models, objective, Data, Model_Params, Train_Params):
    (NUM_EPOCHS, NUM_DATASETS, S, B, CUDA, device, path) = Train_Params
    SubTrain_Params = (device, S, B) + Model_Params
    ##(N, K, D, mcmc_size) = Model_Params
#     indices = torch.randperm(NUM_DATASETS)
    step = 2
    indices = torch.arange(NUM_DATASETS)
    batch_indices = indices[0*B : (0+1)*B]
    obs = Data[batch_indices]
    obs = shuffler(obs).repeat(S, 1, 1, 1)
    if CUDA:
        obs =obs.cuda().to(device)
    loss, metric_step, reused = objective(models, obs, SubTrain_Params)
    return obs, metric_step, reused

In [7]:
BATCH_SIZE_TEST = 50
Train_Params_Test = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE, PATH)
obs, metric_step, reused = test(models, EP, Data, Model_Params, Train_Params_Test)
(q_eta, _, q_z, _, _, _) = reused

ValueError: not enough values to unpack (expected 6, got 4)